This Python notebook takes a subset of POIs in Downtown Santa Cruz, and moves each one to the closest building. The data will be saved to disk as csv and parquet files.

In [ ]:
!pip install pandas --quiet
!pip install geopandas --quiet
!pip install shapely --quiet
!pip install duckdb --quiet
!pip install jupysql --quiet
!pip install duckdb-engine --quiet
!pip install folium matplotlib mapclassify --quiet

In [ ]:
import pandas as pd
import geopandas as gpd
import duckdb
from shapely import wkt
from shapely import Point
from shapely.ops import nearest_points
from shapely.ops import transform
import shapely
from pyproj import Transformer

In [ ]:
%reload_ext sql

In [ ]:
%sql duckdb://

In [ ]:
%sql INSTALL spatial;
%sql INSTALL httpfs;
%sql LOAD spatial;
%sql LOAD httpfs;
%sql SET s3_region='us-west-2'

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
%%sql
COPY(
WITH places AS (
    SELECT
        id,
        names.primary as name,
        geometry
    FROM
        read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=places/type=place/*')
    WHERE
        bbox.xmin BETWEEN -122.077563 AND -121.932679 AND
        bbox.ymin BETWEEN 36.945596 AND 37.007650
),
buildings AS (
    SELECT
        geometry,
        bbox.xmin as xmin,
        bbox.ymin as ymin
    FROM
        read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=buildings/type=building/*')
    WHERE
        bbox.xmin BETWEEN -122.09 AND -121.91 AND
        bbox.ymin BETWEEN 36.93 AND 37.02
)
SELECT
    places.id,
    places.name,
    places.geometry,
    buildings.geometry
FROM places
    LEFT JOIN buildings ON st_intersects(
        st_buffer(places.geometry::geometry, 0.004),
        st_point(buildings.xmin, buildings.ymin)
    )
) TO 'place_buildings.parquet'

In [ ]:
%%sql
COPY(
WITH places AS (
    SELECT
        id,
        names.primary as name,
        geometry
    FROM
        read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=places/type=place/*')
    WHERE
        bbox.xmin BETWEEN -122.077563 AND -121.932679 AND
        bbox.ymin BETWEEN 36.945596 AND 37.007650
),
roads AS (
    SELECT
        geometry,
        class,
        bbox.xmin as xmin,
        bbox.ymin as ymin
    FROM
        read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=transportation/type=segment/*')
    WHERE
        bbox.xmin BETWEEN -122.09 AND -121.91 AND
        bbox.ymin BETWEEN 36.93 AND 37.02 AND (subtype = 'road')
)
SELECT
    places.id,
    roads.geometry,
    roads.class
FROM places
    LEFT JOIN roads ON st_intersects(
        st_buffer(places.geometry::geometry, 0.003),
        st_point(roads.xmin, roads.ymin)
    )
) TO 'place_roads.parquet'

In [ ]:
# Save dataframe as a list of places, where each place has a list of nearby buildings
places_buildings = gpd.read_parquet(path='place_buildings.parquet')

places_buildings = places_buildings.groupby('id', as_index=False).agg({
    'name': 'first',
    'geometry': 'first',
    'geometry_1': list
})

In [ ]:
# Save dataframe as a list of places, where each place has a list of nearby roads
places_roads = gpd.read_parquet(path='place_roads.parquet')

places_roads = places_roads.groupby('id', as_index=False).agg({
    'geometry': list,
    'class': list
})

In [ ]:
places_buildings_roads = pd.merge(places_buildings, places_roads, on='id', how='inner')
places_buildings_roads['crossed'] = [[] for _ in range(len(places_buildings_roads))]
places_buildings_roads['snapped'] = places_buildings_roads['geometry_x']

In [ ]:
# Convert to a coordinate system that represents distance in meters
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
transform_back = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
for i in range(len(places_buildings_roads)):
    cur_point = transform(transformer.transform, places_buildings_roads['geometry_x'][i])
    min_distance = 10000000000000
    min_index = -1
    if places_buildings_roads['geometry_1'][i][0] == None:
        continue
    for j in range(len(places_buildings_roads['geometry_1'][i])):
        cur_building = transform(transformer.transform, places_buildings_roads['geometry_1'][i][j])
        # Check if current building is closer than closest building so far
        if shapely.distance(cur_building, cur_point) < min_distance:
            min_distance = shapely.distance(cur_building, cur_point)
            min_index = j
    if min_index != -1:
        # Move point to closest point on closest building
        places_buildings_roads.loc[i, 'snapped'] = transform(transform_back.transform, nearest_points(transform(transformer.transform, places_buildings_roads['geometry_1'][i][min_index]), cur_point)[0])
        old_to_new_line = shapely.LineString([places_buildings_roads['geometry_x'][i], places_buildings_roads['snapped'][i]])
        # Check roads that the move crosses
        if places_buildings_roads['class'][i][0] != None:
            for m in range(len(places_buildings_roads['class'][i])):
                if shapely.intersects(places_buildings_roads['geometry_y'][i][m], old_to_new_line):
                    places_buildings_roads.at[i, 'crossed'].append(places_buildings_roads['class'][i][m])


In [ ]:
# Convert places data to GeoFrame for getting places in Point format
places_buildings1 = gpd.GeoDataFrame(
    places_buildings_roads.drop(['geometry_1', 'snapped', 'crossed', 'geometry_y', 'class'], axis=1),
    geometry=places_buildings_roads['geometry_x'],
    crs="EPSG:4326"
)

places_buildings2 = gpd.GeoDataFrame(
    places_buildings_roads.drop(['geometry_1', 'geometry_x', 'crossed', 'geometry_y', 'class'], axis=1),
    geometry=places_buildings_roads['snapped'],
    crs="EPSG:4326"
)

In [ ]:
%%sql buildings <<
SELECT
    ST_AsText(geometry) as geometry
FROM
    read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=buildings/type=building/*')
WHERE
    bbox.xmin BETWEEN -122.09 AND -121.91 AND
    bbox.ymin BETWEEN 36.93 AND 37.02

In [ ]:
# Convert buildings data to GeoFrame for working directly with building Polygons
buildings = gpd.GeoDataFrame(
    buildings,
    geometry=buildings['geometry'].apply(wkt.loads),
    crs="EPSG:4326"
)

In [ ]:
# Visualize the data, old location is in red, new location is in green

m = buildings.explore()

places_buildings1.explore(m=m, color="red")
places_buildings2.explore(m=m, color="green")

m

In [ ]:
places_buildings_roads = places_buildings_roads.drop(['geometry_1', 'geometry_y', 'class'], axis=1)
places_buildings_roads['geometry'] = places_buildings_roads['geometry_x'].apply(wkt.dumps)
places_buildings_roads['geometry_updated'] = places_buildings_roads['snapped'].apply(wkt.dumps)
places_buildings_roads = places_buildings_roads.drop(['geometry_x', 'snapped'], axis=1)

In [ ]:
# Take the dataframes and save it
places_buildings_roads.to_parquet(path='santa_cruz_places_building_snap.parquet')
places_buildings_roads.to_csv(path_or_buf='santa_cruz_places_building_snap.csv')